In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
def Slope(X, Y, axis=1):
    sorted_idx = np.argsort(X, axis=axis)
    X = np.take_along_axis(X, sorted_idx, axis=axis)
    Y = np.take_along_axis(Y, sorted_idx, axis=axis)
    if axis==1:
        delta_X = X[:,1:] - X[:,:-1]
        delta_Y = Y[:,1:] - Y[:,:-1]
    if axis==0:
        delta_X = X[1:] - X[:-1]
        delta_Y = Y[1:] - Y[:-1]
    return delta_Y / delta_X

def recovery_curve(x, slope, y0=0.5):
    delta_x = x[1:] - x[:-1]
    delta_y = delta_x*slope
    y = np.array([delta_y[:i].sum() for i in range(1, len(x), 1)]) + y0
    y = np.hstack((np.array([y0]), y))
    return y

IV_data = pd.read_csv('./../../Data/Organized/all/InterpolateData_ChangeNoMuch_0.7.csv', \
                    encoding='Big5', index_col=False)

IV_matrix = np.array(IV_data)
K_num = len(np.where(IV_matrix[0,0] == IV_matrix)[0])
K_closing = np.array(IV_data['履約價(收盤價)'])
K_closing = np.reshape(K_closing, (-1, K_num))
IV_closing = np.array(IV_data['隱含波動率(收盤價)'])
IV_closing = np.reshape(IV_closing, (-1, K_num))
K_IVslope_closing = Slope(X=K_closing, Y=IV_closing, axis=1)

K_settlement = np.array(IV_data['履約價(結算價)'])
K_settlement = np.reshape(K_settlement, (-1, K_num))
IV_settlement = np.array(IV_data['隱含波動率(結算價)'])
IV_settlement = np.reshape(IV_settlement, (-1, K_num))
K_IVslope_settlement = Slope(X=K_settlement, Y=IV_settlement, axis=1)




若$X=[x_1, x_2,\dots x_n     ]$  
則 $TimeSeriesData(X, 30)= [[x_1, x_2,\dots x_{30}],\quad [x_2, x_3,\dots x_{31}]\dots [x_{n-30}, x_{n-29},\dots x_{n-1}]],\quad [x_{31}, x_{32},\dots x_n     ]$  

In [12]:
def TimeSeriesData(X, seq_length):
    X_train = np.zeros((len(X)-seq_length, seq_length, len(X[0])))
    y_train = np.zeros((len(X)-seq_length, len(X[0])))
    for i in range(len(X) - seq_length):
        X_train[i] = X[i:i+seq_length]
        y_train[i] = (X[i+seq_length])
    return np.array(X_train), np.array(y_train)
seq_length = 7
magnification_slope = 10000
IV_matrix_forecast = IV_matrix[seq_length*K_num:]
#收盤價
Inputs_closing_slope, Ouputs_closing = TimeSeriesData(K_IVslope_closing*magnification_slope, seq_length=seq_length)
columns_names_OHLC = ['期貨開盤價', '期貨最高價', '期貨最低價', '期貨收盤價']
column_index_OHLC = [IV_data.columns.get_loc(col) for col in columns_names_OHLC]
Inputs_OHLC = IV_matrix[range(0, len(IV_matrix), K_num)]
Inputs_OHLC = Inputs_OHLC[:,column_index_OHLC]
Inputs_OHLC = np.array([Inputs_OHLC[i:i+seq_length+1] for i in range(len(Inputs_OHLC)-seq_length)])
Inputs_OHLC = Inputs_OHLC.astype(float)

train_size = int(len(Inputs_closing_slope)*0.8)
X_train_closing = Inputs_closing_slope[:train_size]
OHLC_train = Inputs_OHLC[:train_size]
y_train_closing = Ouputs_closing[:train_size]
X_test_closing = Inputs_closing_slope[train_size:]
y_test_closing = Ouputs_closing[train_size:]
IV_matrix_test = IV_matrix_forecast[K_num*train_size:]
OHLC_test = Inputs_OHLC[train_size:]


#結算價
Inputs_settlement, Ouputs_settlement = TimeSeriesData(K_IVslope_settlement*magnification_slope, seq_length=seq_length)
train_size = int(len(Inputs_settlement)*0.8)
X_train_settlement = Inputs_settlement[:train_size]
y_train_settlement = Ouputs_settlement[:train_size]
X_test_settlement = Inputs_settlement[train_size:]
y_test_settlement = Ouputs_settlement[train_size:]


In [13]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, concatenate
from keras.optimizers import Adam

# 定義第一個ANN模型
input_lstm = Input(shape=X_train_settlement.shape[1:])
hidden1 = LSTM(100, return_sequences=True, activation='relu')(input_lstm)
hidden2 = LSTM(100, activation='relu')(hidden1)
output = Dense(10, activation='relu')(hidden2)
output = Dense(y_train_closing.shape[1])(output)

# 定義模型
model = Model(input_lstm, outputs=output)

# 編譯模型

model.compile(loss='mse', optimizer = 'adam', metrics=['mse'])

# 顯示模型結構
#model.summary()



# 訓練模型
model.fit(X_train_settlement, y_train_settlement, epochs=50, \
          batch_size=3000, validation_split=0.2)

Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 6.9920 - mse: 6.9920 - val_loss: 2.1796 - val_mse: 2.1796
Epoch 2/50
1/1 [==============================] - 0s 78ms/step - loss: 6.9112 - mse: 6.9112 - val_loss: 2.1694 - val_mse: 2.1694
Epoch 3/50
1/1 [==============================] - 0s 80ms/step - loss: 6.8809 - mse: 6.8809 - val_loss: 2.1619 - val_mse: 2.1619
Epoch 4/50
1/1 [==============================] - 0s 70ms/step - loss: 6.8624 - mse: 6.8624 - val_loss: 2.1533 - val_mse: 2.1533
Epoch 5/50
1/1 [==============================] - 0s 72ms/step - loss: 6.8440 - mse: 6.8440 - val_loss: 2.1447 - val_mse: 2.1447
Epoch 6/50
1/1 [==============================] - 0s 75ms/step - loss: 6.8287 - mse: 6.8287 - val_loss: 2.1375 - val_mse: 2.1375
Epoch 7/50
1/1 [==============================] - 0s 77ms/step - loss: 6.8152 - mse: 6.8152 - val_loss: 2.1306 - val_mse: 2.1306
Epoch 8/50
1/1 [==============================] - 0s 78ms/step - loss: 6.7977 - mse: 6.7977 - val_l

In [10]:
model.fit(X_train_closing, y_train_closing, epochs=20, batch_size=3000, validation_split=0.2)

Epoch 1/20
1/1 [==============================] - 3s 3s/step - loss: 10.5904 - mse: 10.5904 - val_loss: 5.8060 - val_mse: 5.8060
Epoch 2/20
1/1 [==============================] - 3s 3s/step - loss: 10.5967 - mse: 10.5967 - val_loss: 561.2458 - val_mse: 561.2458
Epoch 3/20
1/1 [==============================] - 3s 3s/step - loss: 164.9325 - mse: 164.9325 - val_loss: 6.6993 - val_mse: 6.6993
Epoch 4/20
1/1 [==============================] - 3s 3s/step - loss: 10.6571 - mse: 10.6571 - val_loss: 5.8402 - val_mse: 5.8402
Epoch 5/20
1/1 [==============================] - 3s 3s/step - loss: 10.6487 - mse: 10.6487 - val_loss: 6.6361 - val_mse: 6.6361
Epoch 6/20
1/1 [==============================] - 3s 3s/step - loss: 12.1170 - mse: 12.1170 - val_loss: 5.8343 - val_mse: 5.8343
Epoch 7/20
1/1 [==============================] - 3s 3s/step - loss: 10.6483 - mse: 10.6483 - val_loss: 5.8801 - val_mse: 5.8801
Epoch 8/20
1/1 [==============================] - 3s 3s/step - loss: 10.7121 - mse: 10.7121

In [14]:
test_loss = model.evaluate(X_test_closing, y_test_closing)
print("Test Loss:", test_loss)

# 查看測試輸出
y_pred_settlement = model.predict(X_test_settlement)



12/12 [==============================] - 0s 2ms/step - loss: 3.0155 - mse: 3.0155
Test Loss: [3.0155186653137207, 3.0155186653137207]
12/12 [==============================] - 0s 2ms/step


In [9]:
columns_names_K = ['履約價(收盤價)', '履約價(結算價)']
column_index_K = [IV_data.columns.get_loc(col) for col in columns_names_K]
ForecastIV = np.zeros(((len(y_pred_settlement), K_num)))
for i in range(len(y_pred_settlement)):
    x = IV_matrix_test[i*K_num:(i+1)*K_num, column_index_K[0]]
    v0 = IV_matrix_test[i*K_num,8]
    ForecastIV[i] = recovery_curve(x, y_pred_settlement[i]/(magnification_slope), y0=v0)
ForecastIV = np.reshape(ForecastIV,(-1,1))
Forecast_matrix  = np.hstack((IV_matrix_test, ForecastIV))
column = np.hstack((IV_data.columns.to_numpy(), np.array(['預測隱含波動率(結算價)'])))
Forecast_Data = pd.DataFrame(data=Forecast_matrix, columns=column)
Forecast_Data.to_csv('./../../Data/forecast/ForecastIV_0.7.csv', index=False, encoding='Big5')